In [29]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
import numpy as np

In [16]:
data = pd.read_csv("master_dataset/processed_data.csv")
list(data.columns)
# drop unwanted features


data = data.drop(['title', 'text', 'text_without_stopwords', 'title_without_stopwords','syllables', 'polarity_category', 'overall_content', 'polarity'], axis=1)
#'Topic 1 Probability', 'Topic 2 Probability', 'Topic 3 Probbility' , 'Topic 4 Probability' ,'Topic 5 Probability',
#'title_word_count', 'title_sentence_count', 'title_average_word_length','title_punctuation_count', 'title_stopwords_count'  
# 'polarity_category_Neutral' , 'polarity_category_Positive'

In [17]:
#dataset is slightly imbalanced so we will perform upsampling to balance the dataset.
data['class'].value_counts()

0    21196
1    17462
Name: class, dtype: int64

In [18]:
list(data.columns)

['class',
 'text_word_count',
 'title_word_count',
 'text_sentence_count',
 'title_sentence_count',
 'text_average_word_length',
 'title_average_word_length',
 'text_punctuation_count',
 'title_punctuation_count',
 'text_stopwords_count',
 'title_stopwords_count',
 'flesch_readability',
 'subjectivity',
 'Topic 1 Probability',
 'Topic 2 Probability',
 'Topic 3 Probbility',
 'Topic 4 Probability',
 'Topic 5 Probability',
 'polarity_category_Neutral',
 'polarity_category_Positive']

In [19]:
data.describe()

,class,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability,polarity_category_Neutral,polarity_category_Positive
count,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000
mean,0.451705,411.374515,12.161209,15.050830,1.067877,5.138347,5.592058,53.119147,1.715635,155.440685,1.136789,0.006674,0.402753,0.190953,0.324903,0.125233,0.185360,0.173552,0.375291,0.523255
std,0.497669,322.881353,3.765686,11.985707,0.270277,1.875575,1.430398,63.542364,1.452448,123.275458,1.295738,1.003826,0.124806,0.281023,0.312565,0.210573,0.258829,0.234861,0.484204,0.499465
min,0.000000,1.000000,1.000000,1.000000,1.000000,2.250000,3.000000,0.000000,0.000000,0.000000,0.000000,-83.616811,0.000000,0.000376,0.000246,0.000120,0.000303,0.000268,0.000000,0.000000
25%,0.000000,220.000000,10.000000,7.000000,1.000000,4.866522,5.000000,26.000000,1.000000,80.000000,0.000000,-0.169506,0.337127,0.002949,0.010969,0.001305,0.003113,0.003024,0.000000,0.000000
50%,0.000000,376.000000,11.000000,13.000000,1.000000,5.075727,5.500000,45.000000,2.000000,142.000000,1.000000,0.050549,0.405745,0.030471,0.231605,0.006765,0.056382,0.056373,0.000000,1.000000
75%,1.000000,522.000000,14.000000,20.000000,1.000000,5.272987,6.066667,67.000000,3.000000,201.000000,2.000000,0.267479,0.475000,0.282926,0.584998,0.157612,0.272065,0.268291,1.000000,1.000000
max,1.000000,8436.000000,45.000000,321.000000,4.000000,149.000000,149.000000,7295.000000,26.000000,3017.000000,15.000000,2.614284,1.000000,0.997819,0.998037,0.996771,0.998911,0.998767,1.000000,1.000000


In [22]:
# Since we will be performing SVC, and SVC works better on scaled data, we will be scaling all our to ensure that the model runs smoothly
# We will use the Min Max scaler

min_max_scaler = MinMaxScaler()

# Since polarity_category_Neutral and polarity_category_Positive are already from 0 to 1 scale, we will not have to further normalize it.
cols = data.iloc[:,:-2].columns

#cols = ['text_word_count','title_word_count','text_sentence_count','title_sentence_count','text_average_word_length',
#        'title_average_word_length','text_punctuation_count','title_punctuation_count',
#        'text_stopwords_count','title_stopwords_count','flesch_readability']

# Normalize the data , since the data is not normally distributed we will use minmaxscaler
data[cols] = min_max_scaler.fit_transform(data[cols]) 



In [24]:
data.describe()

,class,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability,polarity_category_Neutral,polarity_category_Positive
count,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000
mean,0.451705,0.048651,0.253664,0.043909,0.022626,0.019682,0.017754,0.007282,0.065986,0.051522,0.075786,0.969760,0.402753,0.191066,0.325376,0.125533,0.185314,0.173544,0.375291,0.523255
std,0.497669,0.038279,0.085584,0.037455,0.090092,0.012781,0.009797,0.008710,0.055863,0.040860,0.086383,0.011641,0.124806,0.281743,0.313257,0.211281,0.259189,0.235214,0.484204,0.499465
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.025963,0.204545,0.018750,0.000000,0.017830,0.013699,0.003564,0.038462,0.026516,0.000000,0.967717,0.337127,0.002580,0.010747,0.001189,0.002814,0.002760,0.000000,0.000000
50%,0.000000,0.044458,0.227273,0.037500,0.000000,0.019255,0.017123,0.006169,0.076923,0.047067,0.066667,0.970269,0.405745,0.030172,0.231872,0.006667,0.056157,0.056190,0.000000,1.000000
75%,1.000000,0.061766,0.295455,0.059375,0.000000,0.020600,0.021005,0.009184,0.115385,0.066622,0.133333,0.972785,0.475000,0.283274,0.586047,0.158022,0.272140,0.268426,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
#first split the dataset into training and test sets
x = data.iloc[:,1:]
y = data.iloc[:,:1]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state = 1)

#balance x_train with oversampling
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy = 1)
x_train,y_train = oversample.fit_resample(x_train, y_train)
data = pd.concat([x_train,y_train],axis = 1)

#check that train set is oversampled
data['class'].value_counts()


1.0    14879
0.0    14879
Name: class, dtype: int64

Linear SVM

In [28]:
# Linear SVM baseline model

linearSVC = SVC(kernel='linear',random_state=1)
linearSVC.fit(x_train, np.ravel(y_train))
y_pred_linearSVC = linearSVC.predict(x_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_pred_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_pred_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_pred_linearSVC))


Accuracy: 0.9599931022590101
Precision: 0.9758940920766647
Recall: 0.9352395379662942
F1_score: 0.9551344034035969


In [30]:
#hyperparameter tuning with gridsearch for SVM  

grid_params = {
    'C': [0.1,1, 10, 100], 
    'gamma': [1,0.1,0.01,0.001]
    }

scorer = metrics.make_scorer(metrics.f1_score)

gridCV = GridSearchCV(SVC(kernel='linear',random_state=1), param_grid = grid_params, cv = 5, scoring = scorer, n_jobs=-1)

gridCV.fit(x_train,np.ravel(y_train))

print("Best Hyper Parameters: ", gridCV.best_params_)

Best Hyper Parameters:  {'C': 100, 'gamma': 1}


In [31]:
finalSVC = SVC( kernel= 'linear', C= 100, gamma= 1, random_state = 1)
finalSVC.fit(x_train, np.ravel(y_train))
y_pred_finalSVC = finalSVC.predict(x_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_finalSVC))
print("Precision:", metrics.precision_score(y_test, y_pred_finalSVC))
print("Recall:",metrics.recall_score(y_test, y_pred_finalSVC))
print("F1_score:", metrics.f1_score(y_test, y_pred_finalSVC))


Accuracy: 0.9641317468529057
Precision: 0.9746341463414634
Recall: 0.9458435902291232
F1_score: 0.9600230636171438
